### Importance of High-Quality Test Dataset for RAG Systems

### Purpose
Test sets are critical for:
- Accurately measuring RAG system performance
- Identifying system strengths and weaknesses
- Guiding continuous improvement

### Key Evaluation Dimensions
1. **Retrieval Effectiveness**: Assess how well relevant context is retrieved
2. **Generation Quality**: Evaluate the accuracy and coherence of generated responses
3. **Contextual Relevance**: Measure how well the system understands and integrates retrieved information

### Evaluation Goals
- Benchmark system performance
- Detect hallucinations
- Validate generalization capabilities
- Simulate real-world query complexity

### Best Practices
- Use diverse query types
- Cover multiple domains
- Include edge cases
- Create reproducible test scenarios
- Contains enough number of samples to derive statistically significant conclusions

### 0. Configuration for generating the goldens

Goldens are the samples containing the `reference`, `reference_contexts` and `user_input`. For filling out the rest of the fields: `response` and `retrieved_contexts`, I use various different configurations, which are to be found at the root of the project under *experiments.csv*.

The goldens are going to be saved under `./goldens`.

Configuration for the generation of *goldens*:

* For the generation of the goldens for all experiments I will make use of the following parameters:
    
    * Generation model - `llama3.1:8b-instruct-q4_1`

    * Temperature - `0.0`

    * The rest of the parameters will vary depending on the experiments configuration

* For filling out the rest of the fields:

    * Generation model - depends on the experiment configuration

    * Temperature - depends on the experiment configuration

    * The rest of the parameters will vary depending on the experiment configuration

* The goal is that I make use of the stronger instruction following capabilities of `llama3.1:8b-instruct-q4_1` to try to generate a synthetic dataset, which is *clean* and *error-free/logic-free* as possible and to then try the **RAG** application with various settings and compare the results.

* I think that using the same `model` and `temperature` for the generation of *goldens* would make the experiments fair.


### 1. Retrieve data:
* The data can be a dataset from `huggingface` or any other platform.

* Alternatively, files available on disk - pdf, md, etc.

* One can also use `AsyncHtmlLoader` from `langchain` to scrape from the internet.
    - **Careful when performing web scraping to not violate any terms and conditions!**

**NOTE**:

* Make sure you install the requirements first if you want to test the notebook.

    * To do so run the `setup.sh` in the parent folder.

* Make sure you select the proper environment as your kernel: `eval`. 

In [1]:
# For this notebook I will use a dataset provided by RAGAs.
# The dataset contains markdown files about a fictional airline company.
! git clone https://huggingface.co/datasets/explodinggradients/ragas-airline-dataset data

Cloning into 'data'...
remote: Enumerating objects: 14, done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 14 (from 1)
Unpacking objects: 100% (14/14), 16.16 KiB | 5.39 MiB/s, done.


### 2. Load data into document objects

For extracting data from documents and splitting them into chunks one can make use of various frameworks:

* `langchain` <- The one I use

* `llamaindex` <- Another popular solution

Both frameworks provide various abstractions to load documents, extract data and split into chunks.

In [1]:
import os
from typing import Final

from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv("../../env/rag.env")

# The path is the folder, where the documents are stored at.
# Make sure you select the proper one.
DIR_PATH: Final[str] = "data/"
loader = DirectoryLoader(
    DIR_PATH,
    glob="**/*.md",
    exclude="README.md"
)

# The R2R framework uses a recrusive character text splitter to split the documents.
# For that purpose I use it to try to mimic the same behavior.
splitter = RecursiveCharacterTextSplitter(
    chunk_size=int(os.getenv("CHUNK_SIZE")),
    chunk_overlap=int(os.getenv("CHUNK_OVERLAP")),
    length_function=len
)

docs: list[Document] = loader.load_and_split(splitter)

### 3. Construct knowledge graph

- A **knowledge graph** is a fundamental concept when it comes to **RAGAs** and using its capabilities for **automatic synthetic data generation**.

- A **knowledge graph** consists of **Node**s at first, which represent **documents/chunks** - their content and additionally metadata (optional).

- Thereafter, one can enrich the graph by using various **extractors** and applying different **transformations**. Doing so additional attributes get added to the relevant nodes and **relationships can get built**, which express some kind of connection between Node objects. The transformations can be applied only through the use of **Extractor**s, **Splitter**s and or **RelationshipBuilder**s. They serve as a way to gather relevant data from the documents depending on the type of extractor and this way to logically connect 2 or more nodes together.

- Finally, the graph is used to generate so called **Scenario**s and can also be used to generate **Persona**s.

![Knowledge graph creation workflow RAGAs](../../img/kg_rag.webp "Knowledge graph RAGAs")

In [2]:
from ragas.testset.graph import (
    Node,
    NodeType,
    KnowledgeGraph,
)

kg = KnowledgeGraph()

for doc in docs:
    kg.add(
        Node(
            type=NodeType.CHUNK, # Since we already split the documents, we can use the chunk type.
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

## 4. Instantiate required objects

- **RAGAs** would require a **LLM** and an **Embedding model** depending on the type of **Transformation**s one would like to apply to the **Knowledge Graph**. For that purpose one must create *wrapper* objects for both of the models. `langchain` and `llama-index` are both supported. 

- Additionally, a **configuration** can be used to modify the default behaviour of the framework. For example timeout values can be modified, maximum retries for failed operations and so on can be configured from the **RunConfig**.
    - **NOTE**: depending on the LLM model and GPU you may need to modify the `timeout` value, otherwise you will stumble upon `TimeoutException`

- Lastly, there's a single implementation in **RAGAs** for caching intermediate steps onto disk. To use it the **DiskCacheBackend** class can come in play.

In [3]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

from ragas import RunConfig, DiskCacheBackend
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

run_config = RunConfig(
    timeout=86400,    # 24 hours on waiting for a single operation
    max_retries=20,   # Max retries before giving up
    max_wait=600,     # Max wait between retries
    max_workers=4,    # Concurrent requests
    log_tenacity=True # Print retry attempts
)

# This stores data generation and evaluation results locally on disk
# When using it for the first time, it will create a .cache folder
# When using it again, it will read from that folder and finish almost instantly
cacher = DiskCacheBackend(cache_dir=".cache")

langchain_llm = LangchainLLMWrapper(
    langchain_llm=ChatOllama(
        model=os.getenv("DATA_GENERATION_MODEL"),
        base_url="http://localhost:11434",
        temperature=float(os.getenv("DATA_GENERATION_TEMPERATURE")),
        num_ctx=int(os.getenv("LLM_CONTEXT_WINDOW_TOKENS")),
        format="json" # If some operations would require a JSON output for proper extraction
    ),
    run_config=run_config,
    cache=cacher
)

langchain_embeddings = LangchainEmbeddingsWrapper(
    embeddings=OllamaEmbeddings(
        model=os.getenv("EMBEDDING_MODEL"),
        base_url="http://localhost:11434"
    ),
    run_config=run_config,
    cache=cacher
)

### 5. Create the transformation pipeline

The sequence of transformations:

1. Named Entity Recognition (NER) and Keyphrases extraction 
    - NERExtractor identifies and extracts named entities (e.g., people, organizations, locations).  

    - KeyphrasesExtractor extracts the main keyphrases to be found in the text

2. NEROverlapBuilder and KeyphraseOverlapBuilder
    - Used to establish a relationship between nodes containing similar:
        
        - entities
        - keyphrases

3. Parallel Processing for Efficiency:
    - Certain transformations can run in parallel to improve performance.

- Final Outcome:
    - A structured set of document transformations that extract valuable information
    - Used to enrich the knowledge graph for further generation of scenarios and finally samples

**NOTE:** Some of the extractors *(LLM-based ones)* do receive an optional `prompt`, which one can use to modify the workflow. For instance the `NERExtractor` can receive a custom prompt, which could contain instructions that differ from the original one and extracts entities in a different way.

In [4]:
from ragas.testset.transforms import (
    Parallel,
    OverlapScoreBuilder,
    KeyphrasesExtractor,
)
from ragas.testset.transforms.extractors import NERExtractor

from prompts.extractors.custom_ner_prompt import MyNERPrompt
from prompts.extractors.custom_keyphrases_prompt import MyKeyphrasesExtractorPrompt

ner_extractor = NERExtractor(
    llm=langchain_llm,
    prompt=MyNERPrompt(
        name="custom_ner_extractor_prompt"
    ),
    max_num_entities=15
)

keyphrases_extractor = KeyphrasesExtractor(
    llm=langchain_llm,
    prompt=MyKeyphrasesExtractorPrompt(
        name="custom_keyphrases_extractor_prompt"
    ),
    max_num=15
)

ner_overlap_sim = OverlapScoreBuilder()

keyphrases_overlap_sim = OverlapScoreBuilder(
    property_name="keyphrases",
)

transforms = [
    Parallel(
        ner_extractor,
        keyphrases_extractor
    ),
    Parallel(
        ner_overlap_sim,
        keyphrases_overlap_sim
    )
]

### 6. Apply the transformations to the knowledge graph

In the cell below the `apply_transforms` is going to apply all the previously defined transformations enriching the `knowledge graph` in the process.

In [5]:
from ragas.testset.transforms import apply_transforms

apply_transforms(
    kg,
    transforms,
    run_config
)

Applying [NERExtractor, KeyphrasesExtractor]:   0%|          | 0/138 [00:00<?, ?it/s]

Applying [OverlapScoreBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

### 7. Generating personas

- A **Persona** is an entity/role which interacts with the system. **Personas** provide context and perspective, ensuring that **generated queries are natural, user-specific, and diverse**.

- Example: a Senior DevOps engineer, a Junior Data Scientist, a Marketing Manager in the context of an IT company

- **Persona** object consists of a **name** and a **description**.
    
    - The name is used to identify the persona and the description is used to describe the role of the persona.

- Do note that personas can also be generated by a **knowledge graph** if you have one available

In [6]:
from ragas.testset.persona import Persona

# This example is taken from `RAGAs`:
# https://docs.ragas.io/en/latest/howtos/applications/singlehop_testset_gen/#configuring-personas-for-query-generation

persona_first_time_flier = Persona(
    name="First Time Flier",
    role_description="Is flying for the first time and may feel anxious. Needs clear guidance on flight procedures, safety protocols, and what to expect throughout the journey.",
)

persona_frequent_flier = Persona(
    name="Frequent Flier",
    role_description="Travels regularly and values efficiency and comfort. Interested in loyalty programs, express services, and a seamless travel experience.",
)

persona_angry_business_flier = Persona(
    name="Angry Business Class Flier",
    role_description="Demands top-tier service and is easily irritated by any delays or issues. Expects immediate resolutions and is quick to express frustration if standards are not met.",
)

personas: list[Persona] = [
    persona_first_time_flier,
    persona_frequent_flier,
    persona_angry_business_flier
]

## 8. Generate query types 

- There are two main types of queries in **RAGAs**:
    
    - **SingleHopQuery** where the **context** relevant for answering a question lies in a **single document/chunk**

    - **MultiHopQuery** where the **context** relevant for answering a question lies in **multiple documents/chunks**

- Additionally, for each of those queries there's a **Specific** or **Abstract** query variant:
    
    - **Specific** one which pertains to a **fact**. 

        - Example: When did WW1 break out? (Can be precisely answered, there's no room for guessing/interpretation)
    
    - **Abstract** one which is more about testing the **reasoning** capabilities of the LLM. 

        - Example: Why did WW1 break out? (There's room for interpretation in this case)

- **Specific** vs. **Abstract Queries** in a RAG
    - Specific Query: Focuses on clear, fact-based retrieval. The goal in RAG is to retrieve highly relevant information from one or more documents that directly address the specific question.

    - Abstract Query: Requires a broader, more interpretive response. In RAG, abstract queries challenge the retrieval system to pull from documents that contain higher-level reasoning, explanations, or opinions, rather than simple facts.

![Query tpes in RAGAs](../../img/ragas_query_types.png  "Queries")

**Synthesizers** are responsible for **converting enriched nodes and personas into queries**. They achieve this by **selecting a node property (e.g., "entities" or "keyphrases"), pairing it with a persona, style, and query length**, and then using a LLM to generate a query-answer pair based on the content of the node.

* Query lengths may vary:
    - short
    - medium
    - long

* Query style:
    - misspelled
    - websearch-like
    - perfect-grammar
    - poor-grammar

Note that **synthesizers** can additionally be extended/modified by specifying custom **prompts**.


In [7]:
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer

from prompts.synthesizers.custom_themes_matching import MyThemesPersonasMatchingPrompt
from prompts.synthesizers.custom_multi_hop_qa_generation import MyMultiHopQAGenerationPrompt
from prompts.synthesizers.custom_single_hop_qa_generation import MySingleHopQAGenerationPrompt

single_hop_specific_entities = SingleHopSpecificQuerySynthesizer(
    llm=langchain_llm,
    generate_query_reference_prompt=MySingleHopQAGenerationPrompt(),
    theme_persona_matching_prompt=MyThemesPersonasMatchingPrompt(),
    property_name="entities"
)

single_hop_specific_keyphrases = SingleHopSpecificQuerySynthesizer(
    llm=langchain_llm,
    generate_query_reference_prompt=MySingleHopQAGenerationPrompt(),
    theme_persona_matching_prompt=MyThemesPersonasMatchingPrompt(),
    property_name="keyphrases"
)

multi_hop_specific_entities = MultiHopSpecificQuerySynthesizer(
    llm=langchain_llm,
    generate_query_reference_prompt=MyMultiHopQAGenerationPrompt(),
    theme_persona_matching_prompt=MyThemesPersonasMatchingPrompt()
)

multi_hop_specific_keyphrases = MultiHopSpecificQuerySynthesizer(
    llm=langchain_llm,
    generate_query_reference_prompt=MyMultiHopQAGenerationPrompt(),
    relation_type="keyphrases_overlap",
    property_name="keyphrases",
    theme_persona_matching_prompt=MyThemesPersonasMatchingPrompt()
)

query_distribution = [
    (single_hop_specific_entities, 0.25),
    (single_hop_specific_keyphrases, 0.25),
    (multi_hop_specific_entities, 0.25),
    (multi_hop_specific_keyphrases, 0.25)
]

### 9. Generate the samples

#### Definition of evaluation sample

An evaluation sample is a single structured data instance that is used to assess and measure the performance of your LLM application in specific scenarios. It represents a single unit of interaction or a specific use case that the AI application is expected to handle. In Ragas, evaluation samples are represented using the `SingleTurnSample` and `MultiTurnSample` classes.

#### SingleTurnSample

`SingleTurnSample` represents a single-turn interaction between a user, LLM, and expected results for evaluation. It is suitable for evaluations that involve a single question and answer pair, possibly with additional context or reference information.

This type of sample is ideal for straightforward question-answering scenarios where a user asks a single question and expects a direct response.

#### MultiTurnSample

`MultiTurnSample` represents a multi-turn interaction between Human, AI and optionally a Tool and expected results for evaluation. It is suitable for representing conversational agents in more complex interactions for evaluation.

In `MultiTurnSample`, the `user_input` attribute represents a sequence of messages that collectively form a multi-turn conversation between a human user and an AI system. These messages are instances of the classes `HumanMessage`, `AIMessage`, and `ToolMessage`.

This type of sample is designed for evaluating more complex conversational flows where multiple turns of dialogue occur, potentially involving tool usage for gathering additional information.

![Scenario generation workflow RAGAs](../../img/scenario_rag.webp "Scenarios RAGAs")

In [ ]:
from ragas.testset import TestsetGenerator, Testset

generator = TestsetGenerator(
    langchain_llm,
    langchain_embeddings,
    kg,
    personas
)

dataset: Testset = generator.generate(
    testset_size=50,
    query_distribution=query_distribution,
    num_personas=len(personas),
    run_config=run_config,
    with_debugging_logs=True,
)

### 10. Save dataset containing goldens (no actual output and retrieval context)

In [ ]:
# The id should match the experiment_id
goldens_id: str = input("Enter goldens id (Ex. 1): ")

dataset.to_jsonl(f"./goldens/{goldens_id}_goldens.jsonl")

### 11. Upload to the cloud (Optional)

* To upload the data on **app.ragas.io** make sure you:
    * First create an account
    * Get an **API key**
    * Finally, create a `.env` file in the parent folder like so and export it in your notebook:

```bash
RAGAS_APP_TOKEN=apt.1234a-......-9dfew
```

In [10]:
from dotenv import load_dotenv

# Load the token
load_dotenv("../.env") 

dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/2446e75e-efd3-41e8-aacd-bcafbe3ad280


'https://app.ragas.io/dashboard/alignment/testset/2446e75e-efd3-41e8-aacd-bcafbe3ad280'

### 12. Fill out the missing fields in the dataset to complete it.

* This script will first create a virtual environment with all required dependencies.

* Next the files that are currently ingested will be removed.

* The files from the `data` folder will be re-ingested.

* Then the dataset will be filled out with `response` and `retrieved contexts`.

* Finally, the dataset will be persisted into `/datasets`.

This is required since the experiments have varying `chunk_size` and `chunk_overlap` values. Because of that the application needs to be restarted each time and the data ingested newly, so we can carry out the data generation properly.

**NOTE**:

- Make sure you set the proper values under `/env/rag.env` relative to the experiment id you want to carry out.

- For each experiment provide an intuitive name to distinguish between the experiments

    - In my case I just use the experiment id

In [9]:
# Make the script executable
!chmod u+x ./fill_dataset.sh

In [3]:
goldens_filepath: str = f"{goldens_id}_goldens"
test_id: str = f"{goldens_id}"

# Run the script with arguments
# Also do provide different names for the different configurations
!./fill_dataset.sh {goldens_filepath} {test_id}

[+] VIRTUAL ENVIRONMENT ALREADY EXISTS. SKIPPING CREATION. [+]
[+] INSTALLING DEPENDENCIES... [+]
  Using cached r2r-3.5.11-py3-none-any.whl.metadata (9.3 kB)
  Using cached langchain_community-0.3.23-py3-none-any.whl.metadata (2.5 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached tiktoken-0.8.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached python_json_logger-3.3.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached aioshutil-1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached aiosqlite-0.20.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached APScheduler-3.11.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached asyncpg-0.29.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached azure_ai_inference-1.0.0b9-py3-none-any.whl.metadata (34 kB)
  

### 13. Synthetic Data Generation completed

With this you have created your own synthetic goldens, used various configurations to fill out the rest of the missing fields and saved all the data locally. Now you can move on to the next notebook, check out all the relevant (in my opinion) metrics and evaluate your application using the different experiments.